<a href="https://colab.research.google.com/github/AgamjotKC/Brainwave_Matrix_Intern/blob/main/fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded_fake = files.upload()


In [ ]:
from google.colab import files
uploaded_true = files.upload()

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
#loading datasets
fake_df = pd.read_csv('Fake.csv')
true_df = pd.read_csv('True.csv')

In [ ]:
#labelling the dataset
fake_df["Label"] = 0
true_df["Label"] = 1


true_df.tail()

,title,text,subject,date,Label
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [ ]:
#concatenating dataset
data = pd.concat( [fake_df, true_df], ignore_index = True)

#shuffling the dataset
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,title,text,subject,date,Label
0,Burundi takes steps to extend president's rule...,NAIROBI (Reuters) - Burundi s cabinet backed a...,worldnews,"October 27, 2017",1
1,Horrific Texas GOP Platform Is Made Worse By ...,When you live in a state that thinks the worst...,News,"May 19, 2016",0
2,California poised to shut out Republicans from...,"SACRAMENTO, Calif. (Reuters) - California vote...",politicsNews,"June 7, 2016",1
3,CAN HILLARY LIE HER WAY OUT OF THIS ONE? PHYSI...,Will Hillary s health be the only thing she ca...,politics,"Aug 12, 2016",0
4,Scotland's Sturgeon sees no reason not to get ...,EDINBURGH (Reuters) - An agreement to keep Nor...,worldnews,"December 4, 2017",1


In [ ]:
#merging text and title column
data['text_combined'] = data['title'].astype(str) + " " + data['text'].astype(str)

#cleaning data function
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)    # Remove HTML tags
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove non-alphabetic characters
    text = re.sub(r'\s+', ' ', text)     # Remove extra whitespace
    return text.strip().lower()

# Clean the text
data['text_combined'] = data['text_combined'].apply(clean_text)

In [ ]:
#verifying columns
print("Columns in the DataFrame:", data.columns)

In [ ]:
#splitting data
X_train, X_test, y_train, y_test = train_test_split(data['text_combined'], data['Label'], test_size=0.2, random_state=42)

# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [ ]:
# Model initialization and training
model = MultinomialNB()
model.fit(X_train, y_train)

# Predicting on test data
y_pred = model.predict(X_test)


In [ ]:
# Model evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print(classification_report(y_test, y_pred))


Accuracy: 0.93
              precision    recall  f1-score   support

           0       0.93      0.94      0.93      4692
           1       0.93      0.92      0.93      4288

    accuracy                           0.93      8980
   macro avg       0.93      0.93      0.93      8980
weighted avg       0.93      0.93      0.93      8980



In [ ]:
# Confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_mat)

In [ ]:
# Step 6: Test the model with custom input
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Function to clean text
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)    # Remove HTML tags
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove non-alphabetic characters
    text = re.sub(r'\s+', ' ', text)     # Remove extra whitespace
    return text.strip().lower()

# Function to predict fake news
def predict_fake_news(model, text, vectorizer):
    cleaned_text = clean_text(text)
    text_vectorized = vectorizer.transform([cleaned_text])
    prediction = model.predict(text_vectorized)
    return prediction[0]

# Example of using the function
text_to_test = """
    Breaking news! Narendra Modi has not been re-elected as Prime minister.
"""


# Predict
prediction = predict_fake_news(model, text_to_test, vectorizer)

# Output
if prediction == 1:
    print("The text is predicted as FAKE news.")
else:
    print("The text is predicted as TRUE news.")